In [43]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os

H_leaf_dir = Path('/User/kashyappatel/Desktop/ce476-ml-deployment/dataset/healthy')
D_leaf_dir = Path('/User/kashyappatel/Desktop/ce476-ml-deployment/dataset/diseased')

In [44]:
import tensorflow as tf
from tensorflow import keras

In [45]:
image_size = 220
batch_size = 32


target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)


In [46]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)


train_generator = train_datagen.flow_from_directory(
        '/User/kashyappatel/Desktop/ce476-ml-deployment/dataset', 
        target_size=(200, 200),
        batch_size = batch_size,
        
        classes = ['healthy','diseased'],
       
        class_mode='categorical')

Found 0 images belonging to 2 classes.


In [47]:
for image_batch, label_batch in train_generator:
  break
  image_batch.shape, label_batch.shape

In [48]:
print(train_generator.class_indices)

{'healthy': 0, 'diseased': 1}


In [49]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 200x 200 with 3 bytes color
    # The first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(200, 200, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a dense layer
    tf.keras.layers.Flatten(),
    # 128 neuron in the fully-connected layer
    tf.keras.layers.Dense(128, activation='relu'),
    # 5 output neurons for 2 classes with the softmax activation
    tf.keras.layers.Dense(2, activation='softmax')
])

In [50]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 198, 198, 16)      448       
                                                                 
 max_pooling2d_15 (MaxPooli  (None, 99, 99, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_16 (Conv2D)          (None, 97, 97, 32)        4640      
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 48, 48, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 46, 46, 64)        18496     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 23, 23, 64)       

In [51]:
from keras.optimizers.legacy import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

In [52]:
total_sample=train_generator.n

In [53]:
n_epochs = 5

In [54]:
total_sample

0

In [55]:
print(len(train_generator))

0


In [56]:
history = model.fit(
        train_generator, 
        steps_per_epoch=int(total_sample/batch_size),  
        epochs=n_epochs,
        verbose=1)
# model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=n_epochs)

ValueError: Unexpected value for `steps_per_epoch`. Received value is 0. Please check the docstring for `model.fit()` for supported values.

In [ ]:
# from keras.layers import Activation, Dense
# model.add(Dense(3, activation='softmax'))

In [ ]:
model.save('HD_Model.h5')

/Users/kashyappatel/Desktop/ce476-ml-deployment/deploy/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.save(filepath="save_model/")

INFO:tensorflow:Assets written to: save_model/assets


INFO:tensorflow:Assets written to: save_model/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('save_model')
tflite_model = converter.convert()
open("leaves_model.tflite", "wb").write(tflite_model)

2023-10-21 06:47:14.755667: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-10-21 06:47:14.755687: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-10-21 06:47:14.756729: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: save_model
2023-10-21 06:47:14.757738: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-10-21 06:47:14.757743: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: save_model
2023-10-21 06:47:14.759733: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2023-10-21 06:47:14.760736: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-10-21 06:47:14.808504: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: save_model
2023-10-21 06:47:14.81913

919732

In [ ]:
# import os
# os.remove("./leaves_model.tflite")

In [ ]:
MODEL_NAME = model
def export_model(saver, model, input_node_names, output_node_name):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

In [ ]:
export_model(tf.compat.v1.train.Saver(), model,["conv2d_1_input"], "dense_2/Softmax"H)
# tf.compat.v1.train.Saver()

SyntaxError: invalid syntax (1245273478.py, line 1)

In [ ]:
from tensorflow.keras import Sequential
model.export('leaves_model.tflite', 'leavesLabels.txt')
#  model.export(export_dir='./',tflite_filename='leaves_model.tflite', label_filename='leavesLabels.txt')